In [ ]:
import numpy as np
import pandas as pd

import statistics
import random

from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.over_sampling import SMOTE

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import fbeta_score
from sklearn import metrics

import statsmodels.api as sm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

In [ ]:
q3features_trainvalid = pd.read_pickle("q3features_trainvalid.pickle")
q3target_trainvalid = pd.read_pickle("q3target_trainvalid.pickle")
q3features_test = pd.read_pickle("q3features_test.pickle")
q3target_test = pd.read_pickle("q3target_test.pickle")

Attributes with zero variance were removed from the model; these attributes did not contribute any information to the model.

In [ ]:
#Removing zero variance attributes (training set)
q3features_trainvalid = q3features_trainvalid.loc[:, q3features_trainvalid.nunique(axis=0) != 1]

In [ ]:
#Removing zero variance attributes (test set)
q3features_test = q3features_test.loc[:, q3features_test.nunique(axis=0) != 1]

Attributes with more than 10% missing data were dropped from the dataset. Dr. Iris Eekhout and colleagues in their 2013 paper titled "Missing data in a multi-item instrument were best handled by multiple imputation at the item score level" (https://doi.org/10.1016/j.jclinepi.2013.09.009) found that single-imputation methods can result in biased estimates when the dataset has a higher proportion of missing data. Single-imputation methods will be applied later in the analysis, hence the removal of these attributes with a high proportion of missing data at this step.

In [ ]:
#Removing attributes with more than 10% missing data (training set)
for col in q3features_trainvalid.columns.tolist():
  if q3features_trainvalid[col].isna().sum()/len(q3features_trainvalid[col]) > 0.10:
    q3features_trainvalid = q3features_trainvalid.drop([col], axis=1)

In [ ]:
#Removing attributes with more than 10% missing data (test set)
for col in q3features_test.columns.tolist():
  if q3features_test[col].isna().sum()/len(q3features_test[col]) > 0.10:
    q3features_test = q3features_test.drop([col], axis=1)

Imputation of missing data was performed. The alternative was to use complete case analysis, which involves removing from the dataset any rows that were missing any attributes. Complete case analysis leads to a loss of information and can lead to bias if the missing data is not missing completely at random.

Single-imputation methods were used; missing categorical values were imputed using the mode of that attribute and missing numeric values were imputed using the median. The test data set was imputed using the mode and median values of the training data set. To avoid data leakage, the training data set was not imputed using test set data.

In [ ]:
#Imputing test set
for col in q3features_test.columns.tolist():
  q3features_test[col] = q3features_test[col].replace([None], np.nan)
  if (q3features_test[col].dtype == 'category' or q3features_trainvalid[col].dtype =='datetime64[ns]'):
    q3features_test[col]= q3features_test[col].fillna(random.choice(statistics.multimode(q3features_trainvalid[col])))
  if (q3features_test[col].dtype == 'Int64' or q3features_test[col].dtype == 'int64'):
    q3features_test[col] = q3features_test[col].astype('float64')
  if (q3features_test[col].dtype == 'float64'):
    q3features_test[col] = q3features_test[col].fillna(q3features_trainvalid[col].median())

In [ ]:
#Imputing training set
for col in q3features_trainvalid.columns.tolist():
  q3features_trainvalid[col] = q3features_trainvalid[col].replace([None], np.nan)
  if (q3features_trainvalid[col].dtype == 'category' or q3features_trainvalid[col].dtype =='datetime64[ns]'):
    q3features_trainvalid[col]= q3features_trainvalid[col].fillna(random.choice(statistics.multimode(q3features_trainvalid[col])))
  if (q3features_trainvalid[col].dtype == 'Int64' or q3features_trainvalid[col].dtype == 'int64'):
    q3features_trainvalid[col] = q3features_trainvalid[col].astype('float64')
  if (q3features_trainvalid[col].dtype == 'float64'):
    q3features_trainvalid[col] = q3features_trainvalid[col].fillna(q3features_trainvalid[col].median())

Numeric attributes were standardized; variable importance will be assessed using regression models, and standardization allows for easier interpretation of the model coefficients. Standardization was performed instead of normalization, as standardization is more resistant to outliers.

In [ ]:
#Standardization  of test set
for col in q3features_test.columns.tolist():
  if (q3features_test[col].dtype == 'Int64' or q3features_test[col].dtype == 'int64' or q3features_test[col].dtype == 'float64'):
    for i in range(len(q3features_test[col])):
      q3features_test.loc[q3features_test.index[i], col] = (q3features_test.loc[q3features_test.index[i], col] - q3features_trainvalid[col].mean())/np.std(q3features_trainvalid[col])

In [ ]:
#Standardization of training set
for col in q3features_trainvalid.columns.tolist():
  if (q3features_trainvalid[col].dtype == 'Int64' or q3features_trainvalid[col].dtype == 'int64' or q3features_trainvalid[col].dtype == 'float64'):
    for i in range(len(q3features_trainvalid[col])):
      q3features_trainvalid.loc[q3features_trainvalid.index[i], col] = (q3features_trainvalid.loc[q3features_trainvalid.index[i], col] - q3features_trainvalid[col].mean())/np.std(q3features_trainvalid[col])

In [ ]:
#The SMOTE algorithm cannot handle datetime, so converting to number of
#days since January 1st, year one
for i in range(len(q3features_trainvalid['INTERVIEWDATE'])):
  q3features_trainvalid.loc[q3features_trainvalid.index[i], 'INTERVIEWDATE'] =  q3features_trainvalid.loc[q3features_trainvalid.index[i], 'INTERVIEWDATE'].toordinal()

for i in range(len(q3features_test['INTERVIEWDATE'])):
  q3features_test.loc[q3features_test.index[i], 'INTERVIEWDATE'] =  q3features_test.loc[q3features_test.index[i], 'INTERVIEWDATE'].toordinal()

q3features_trainvalid['INTERVIEWDATE'] = q3features_trainvalid['INTERVIEWDATE'].astype('float64')
q3features_test['INTERVIEWDATE'] = q3features_test['INTERVIEWDATE'].astype('float64')

In [ ]:
#Converting categorical variables to dummy variables
for col in q3features_trainvalid.columns.tolist():
  if (q3features_trainvalid[col].dtype == 'category'):
    q3features_trainvalid = pd.concat([q3features_trainvalid, pd.get_dummies(q3features_trainvalid[col], prefix=col, drop_first=True, dtype='float')], axis=1)
    q3features_trainvalid = q3features_trainvalid.drop([col], axis=1)

In [ ]:
for col in q3features_test.columns.tolist():
  if (q3features_test[col].dtype == 'category'):
    q3features_test = pd.concat([q3features_test, pd.get_dummies(q3features_test[col], prefix=col, drop_first=True, dtype='float')], axis=1)
    q3features_test = q3features_test.drop([col], axis=1)

In [ ]:
q3target_trainvalid = q3target_trainvalid.astype('bool')
q3target_trainvalid = q3target_trainvalid.astype('float')

In [ ]:
q3target_test = q3target_test.astype('bool')
q3target_test = q3target_test.astype('float')

Multicollinearity can lead to model overfitting and can artificially hide the importance of explanatory variables. Multicollinearity (and collinearity) can be assessed using variance inflation factor values. A VIF value of over 10 indicates serious multicollinearity (Vittinghoff, E., Shiboski, S., Glidden, D., & McCulloch, C. (2004). Regression Methods in Biostatistics: Linear, Logistic, Survival and Repeated Measures Models. New York:Springer. https://doi.org/10.1007/b138825). Independent variables were iteratively removed from the model starting with the variable with the highest VIF value until all VIF values were below an accceptable threshold.

In [ ]:
#Removing highly correlated variables

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('STATE')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('_AGE65YR')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('QSTVER')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('IYEAR')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('INTERVIEWDATE')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('DISPCODE')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('SMOKE100_')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('EDUCA')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('HTM4')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('USENOW')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('SEQNO')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('FMONTH')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('_AGE_G')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_trainvalid.columns[q3features_trainvalid.columns.str.startswith('PRIMINSR')]
q3features_trainvalid.drop(unwanted, axis=1, inplace=True)

In [ ]:
unwanted = q3features_test.columns[q3features_test.columns.str.startswith('STATE')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('_AGE65YR')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('QSTVER')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('IYEAR')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('INTERVIEWDATE')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('DISPCODE')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('SMOKE100_')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('EDUCA')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('HTM4')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('USENOW')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('SEQNO')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('FMONTH')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('_AGE_G')]
q3features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q3features_test.columns[q3features_test.columns.str.startswith('PRIMINSR')]
q3features_test.drop(unwanted, axis=1, inplace=True)

Feature selection can help reduce model overfitting. Feature selection can also result in dimensionality reduction, which can improve computational efficiency. Here, a decision tree model was used to evaluate feature importance; this model has a built-in method to compute Gini importance values for each attribute. Gini importance is also referred to as "mean decrease impurity" and is a measure of how a given attribute improves the purity of a node. Attributes with a Gini importance value of less than a robust threshold of 0.01 were removed from the model. Note that dummy variables generated using one-hot encoding will be grouped together; if one variable meets the threshold then all of the set of dummy variables will remain in the model. This is because the entire set of dummy variables is required to represent one categorical variable.

In [ ]:
clf = DecisionTreeClassifier(max_depth=16, random_state=8)
clf.fit(q3features_trainvalid, q3target_trainvalid)
y_pred = clf.predict(q3features_test)

importances = clf.feature_importances_
threshold = 0.01
selected_features = q3features_trainvalid.columns[importances > threshold]
selected_features.tolist()

In [ ]:
filter_col = [col for col in q3features_trainvalid if col.startswith('') or col.startswith('')]

In [ ]:
q3features_trainvalid = q3features_trainvalid[filter_col]
q3features_test = q3features_test[filter_col]

The dataset is imbalanced; a majority of respondents reported that they did not have COVID-19 symptoms that lasted longer than 3 months. Imbalanced data adversely affects model performance; if the model encounters few instances of the minority class then it will be unable to effectively learn from this class.

The SMOTE algorithm was used to oversample the minority class; this address the issue of imbalanced data.

In [ ]:
#SMOTE Algorithm
from imblearn.over_sampling import SMOTE
smo = SMOTE(random_state = 2, k_neighbors=10)
q3features_trainvalid_SM, q3target_trainvalid_SM = smo.fit_resample(q3features_trainvalid, q3target_trainvalid.ravel())

1. Running models using hyperparameters chosen to optimize F-2 Score

Logistic Regression

In [ ]:
logReg = sm.Logit(q3target_trainvalid_SM.ravel(), q3features_trainvalid_SM,).fit()
logRegPrediction = logReg.predict(q3features_test)
logRegPrediction = np.where(logRegPrediction > 0.1, 1, 0)

f2_score = fbeta_score(q3target_test, logRegPrediction, beta=2)
accuracy =  metrics.accuracy_score(q3target_test, logRegPrediction)
recall =  metrics.recall_score(q3target_test, logRegPrediction)
precision = metrics.precision_score(q3target_test, logRegPrediction)

In [ ]:
f2_score
#0.4212405056496416

In [ ]:
accuracy
#0.12733233496579915

In [ ]:
recall
#0.9996815286624203

In [ ]:
precision
#0.12709016559374872

In [ ]:
logRegMatrix = metrics.confusion_matrix(q3target_test, logRegPrediction)
logRegMatrix
#array([[    7, 21560],
#     [    1,  3139]])

Naive Bayes

In [ ]:
model = GaussianNB()
nbModel = model.fit(q3features_trainvalid_SM, q3target_trainvalid_SM)
nbPrediction = (nbModel.predict_proba(q3features_test)[:,1] >= 0.1).astype(bool)

f2_score = fbeta_score(q3target_test, nbPrediction, beta=2)
accuracy =  metrics.accuracy_score(q3target_test, nbPrediction)
recall =  metrics.recall_score(q3target_test, nbPrediction)
precision = metrics.precision_score(q3target_test, nbPrediction)

In [ ]:
f2_score
#0.578369140625

In [ ]:
accuracy
#0.7441210992836038

In [ ]:
recall
#0.7544585987261146

In [ ]:
precision
#0.2991161616161616

In [ ]:
nbMatrix = metrics.confusion_matrix(q3target_test, nbPrediction)
nbMatrix
#array([[16016,  5551],
#       [  771,  2369]])


Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=25, max_depth=4, random_state=42)
rf.fit(q3features_trainvalid_SM, q3target_trainvalid_SM)
#rfPrediction = rf.predict(q3features_trainvalid)
rfPrediction = (rf.predict_proba(q3features_test)[:,1] >= 0.3).astype(bool)

f2_score = fbeta_score(q3target_test, rfPrediction, beta=2)
accuracy =  metrics.accuracy_score(q3target_test, rfPrediction)
recall =  metrics.recall_score(q3target_test, rfPrediction)
precision = metrics.precision_score(q3target_test, rfPrediction)

In [ ]:
f2_score
#0.5801430190982819

In [ ]:
accuracy
#0.6684340470312057

In [ ]:
recall
#0.847452229299363

In [ ]:
precision
#0.25650665124349337

In [ ]:
rfMatrix = metrics.confusion_matrix(q3target_test, rfPrediction)
rfMatrix
#array([[13854,  7713],
#       [  479,  2661]])


Light GBM

In [ ]:
train_data = lgb.Dataset(q3features_trainvalid_SM, label=q3target_trainvalid_SM)
test_data = lgb.Dataset(q3features_test, label=q3target_test, reference=train_data)

params = {
    "num_leaves": 7,
    "reg_alpha": 0.1,
    "reg_lambda": 0.5,
    "min_data_in_leaf": 30,
    "objective": "binary",
    "metric": "binary_logloss",
    "learning_rate": 0.05,
    "force_row_wise": True,
    "bagging_fraction": 0.8,
    "feature_fraction": 0.8
    }
num_round=500
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])
y_pred = bst.predict(q3features_test)
y_pred_binary = (y_pred > 0.1).astype(int)

f2_score = fbeta_score(q3target_test, y_pred_binary, beta=2)
accuracy =  metrics.accuracy_score(q3target_test, y_pred_binary)
recall =  metrics.recall_score(q3target_test, y_pred_binary)
precision = metrics.precision_score(q3target_test, y_pred_binary)

In [ ]:
f2_score
#0.5354449472096531

In [ ]:
accuracy
#0.5799570971789372

In [ ]:
recall
#0.859235668789809

In [ ]:
precision
#0.21355073610891245

In [ ]:
bstMatrix = metrics.confusion_matrix(q3target_test, y_pred_binary)
bstMatrix
#array([[11631,  9936],
#       [  442,  2698]])


2. Running models using hyperparameters chosen to optimize accuracy

Logistic Regression

In [ ]:
logReg = sm.Logit(q3target_trainvalid_SM.ravel(), q3features_trainvalid_SM,).fit()
logRegPrediction = logReg.predict(q3features_test)
logRegPrediction = np.where(logRegPrediction > 0.5, 1, 0)

f2_score = fbeta_score(q3target_test, logRegPrediction, beta=2)
accuracy =  metrics.accuracy_score(q3target_test, logRegPrediction)
recall =  metrics.recall_score(q3target_test, logRegPrediction)
precision = metrics.precision_score(q3target_test, logRegPrediction)

In [ ]:
f2_score
#0.5249466123082897

In [ ]:
accuracy
#0.5577366738171369

In [ ]:
recall
#0.8611464968152867

In [ ]:
precision
#0.20492610837438424

In [ ]:
logRegMatrix = metrics.confusion_matrix(q3target_test, logRegPrediction)
logRegMatrix
#array([[11076, 10491],
#       [  436,  2704]])


Naive Bayes

In [ ]:
model = GaussianNB()
nbModel = model.fit(q3features_trainvalid_SM, q3target_trainvalid_SM)
nbPrediction = (nbModel.predict_proba(q3features_test)[:,1] >= 0.6).astype(bool)

f2_score = fbeta_score(q3target_test, nbPrediction, beta=2)
accuracy =  metrics.accuracy_score(q3target_test, nbPrediction)
recall =  metrics.recall_score(q3target_test, nbPrediction)
precision = metrics.precision_score(q3target_test, nbPrediction)

In [ ]:
f2_score
#0.5706064390032032

In [ ]:
accuracy
#0.8059254462298134

In [ ]:
recall
#0.6694267515923567

In [ ]:
precision
#0.3587642942481652

In [ ]:
nbMatrix = metrics.confusion_matrix(q3target_test, nbPrediction)
nbMatrix
#array([[17810,  3757],
#       [ 1038,  2102]])


Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(q3features_trainvalid_SM, q3target_trainvalid_SM)
#rfPrediction = rf.predict(q3features_trainvalid)
rfPrediction = (rf.predict_proba(q3features_test)[:,1] >= 0.5).astype(bool)

f2_score = fbeta_score(q3target_test, rfPrediction, beta=2)
accuracy =  metrics.accuracy_score(q3target_test, rfPrediction)
recall =  metrics.recall_score(q3target_test, rfPrediction)
precision = metrics.precision_score(q3target_test, rfPrediction)

In [ ]:
f2_score
#0.5348037107575105

In [ ]:
accuracy
#0.8633990367102441

In [ ]:
recall
#0.5544585987261147

In [ ]:
precision
#0.4683884853376379

In [ ]:
rfMatrix = metrics.confusion_matrix(q3target_test, rfPrediction)
rfMatrix
#array([[19591,  1976],
#       [ 1399,  1741]])

Light GBM

In [ ]:
train_data = lgb.Dataset(q3features_trainvalid_SM, label=q3target_trainvalid_SM)
test_data = lgb.Dataset(q3features_test, label=q3target_test, reference=train_data)

params = {
    "num_leaves": 31,
    "reg_alpha": 0.1,
    "reg_lambda": 0.1,
    "min_data_in_leaf": 5,
    "objective": "binary",
    "metric": "binary_logloss",
    "learning_rate": 0.05,
    "force_row_wise": True,
    "bagging_fraction": 0.8,
    "feature_fraction": 0.8
    }
num_round=500
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])
y_pred = bst.predict(q3features_test)
y_pred_binary = (y_pred > 0.5).astype(int)

f2_score = fbeta_score(q3target_test, y_pred_binary, beta=2)
accuracy =  metrics.accuracy_score(q3target_test, y_pred_binary)
recall =  metrics.recall_score(q3target_test, y_pred_binary)
precision = metrics.precision_score(q3target_test, y_pred_binary)

In [ ]:
f2_score
#0.40325784664282877

In [ ]:
accuracy
#0.8686202290848747

In [ ]:
recall
#0.38789808917197455

In [ ]:
precision
#0.47915027537372146

In [ ]:
bstMatrix = metrics.confusion_matrix(q3target_test, y_pred_binary)
bstMatrix
#array([[20243,  1324],
#       [ 1922,  1218]])

Determining Feature Importance of Optimal Model

In [ ]:
from sklearn.inspection import permutation_importance
model = GaussianNB()
nbModel = model.fit(q3features_trainvalid_SM, q3target_trainvalid_SM)
imps = permutation_importance(nbModel, q3features_test, q3target_test)
feature_names=nbModel.feature_names_in_
importances = imps.importances_mean
feature_imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values('Importance', ascending=False)
print(feature_imp_df)

In [ ]:
#      Feature        Importance
#9   DIFFALON_True    0.039220
#17    EMPLOY1_8.0    0.024981
#8      BLIND_True    0.018545
#10  DIFFWALK_True    0.006881
#12    EMPLOY1_3.0    0.003554
#18  MEDCOST1_True    0.003537
#13    EMPLOY1_4.0    0.003181
#19  MENTHLTH_True    0.003076
#16    EMPLOY1_7.0    0.001036
#3        PHYSHLTH    0.000219
#4        SLEPTIM1    0.000040
#14    EMPLOY1_5.0    0.000024
#1        CPDEMO1C    0.000008
#0        CHILDREN   -0.000130
#6          _AGE80   -0.000146
#2        DROCDY4_   -0.000170
#5           WTKG3   -0.000267
#7   ADDEPEV3_True   -0.000510
#11    EMPLOY1_2.0   -0.000793
#15    EMPLOY1_6.0   -0.001036